In [ ]:
import logging
 
def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])
 
    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)
 
    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)
 
    return logger

# Loading Data
import numpy as np
import math
import random
import copy
import tqdm
import torch
import torch.nn as nn
import gpytorch
from matplotlib import pyplot as plt
import pandas as pd

# Make plots inline
%matplotlib inline
import urllib.request
import os
from scipy.io import loadmat
from math import floor
from utils.tools import StandardScaler

#test Razvan's model
from models.ours import PA
import warnings

warnings.filterwarnings('ignore')

def setup_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
# set seed
setup_seed(42)

# this is for running the notebook in our testing framework
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
num_nodes = 1

whole_data = np.load('./data/ACSF1.npy', allow_pickle=True) ##
#["Yearly", "Quarterly", "Monthly", "Weekly", "Daily", "Hourly"]
exp_name = "0" #0-9   4-11 has outlier
pre_horizon = 12
seq_len, pred_len = pre_horizon, 1
results = np.zeros([20,3], dtype=np.float32)
order = 0
output_dim = 1

for item in whole_data.item():
    name_now = ""
    for char in item:
        if char == '_':
            break
        name_now += char
    if exp_name == name_now:
        data = whole_data.item()[item]
        data = np.array(data, dtype=np.float32)
        train_end = int((70. / 100.) * len(data))
        test_end = int((90. / 100.) * len(data))
        train_data = data[:train_end]
        mean = np.mean(train_data)
        std = np.std(train_data)
        data = (data - mean)/std 
        data = torch.from_numpy(data)
        train_data = torch.cat((torch.zeros((seq_len-1,)),data[:train_end]), axis=0)
        valid_data = torch.cat((torch.zeros((seq_len-1,)),data[test_end:]), axis=0)
        test_data = torch.cat((torch.zeros((seq_len-1,)),data[train_end:test_end]), axis=0)
        train_data = train_data.unfold(0, seq_len+pred_len, 1).unsqueeze(-1)
        valid_data = valid_data.unfold(0, seq_len+pred_len, 1).unsqueeze(-1)
        test_data = test_data.unfold(0, seq_len+pred_len, 1).unsqueeze(-1)
        train_x = train_data[:,:seq_len]
        train_y = train_data[:,seq_len:].squeeze()
        test_x = test_data[:-pre_horizon+1,:seq_len]
        test_y = test_data[:-pre_horizon+1,seq_len:].squeeze()
        val_x = valid_data[:,:seq_len]
        val_y = valid_data[:,seq_len:].squeeze()
        test = test_data

        if torch.cuda.is_available():
            train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()
            test = test.cuda()
            val_x, val_y = val_x.cuda(), val_y.cuda()
            
        print(train_x.shape, train_y.shape, test_x.shape, test_y.shape, val_x.shape, val_y.shape, test.shape)
    
        feature_extractor = PA(horizon=pred_len, lag=seq_len, dynamic=False, supports=None,
                               patch_sizes=[1], channels=32, num_nodes=num_nodes, 
                               input_dim=1, output_dim=1, device='cuda:0')

        ##########################################################################################
        
        base_kernel_set = [gpytorch.kernels.ScaleKernel(gpytorch.kernels.LinearKernel()),\
                           gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel()),\
                           gpytorch.kernels.ScaleKernel(gpytorch.kernels.RQKernel()),\
                           gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())]
        R = 1
        
        final_kernel = 0
        for i in range(R):
            #this_iter = base_kernel_set.copy()
            add_kernel = copy.deepcopy(base_kernel_set[0])
            for j in range(1,len(base_kernel_set)):
                add_kernel += copy.deepcopy(base_kernel_set[j])
            add_kenrel = gpytorch.kernels.ScaleKernel(add_kernel)

            if i == 0:
                mul_kernel = copy.deepcopy(add_kernel)
                final_kernel = copy.deepcopy(mul_kernel)
                continue
            else:
                mul_kernel *= copy.deepcopy(add_kernel)
                final_kernel += copy.deepcopy(mul_kernel)
        
        
        print(final_kernel)
    
        '''
        final_kernel =  gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel()) + \
                        gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()) + \
                        gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel()*gpytorch.kernels.PeriodicKernel()) + \
                        gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()*gpytorch.kernels.PeriodicKernel()) + \
                        gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()*gpytorch.kernels.RBFKernel())
        '''
        ##########################################################################################
        setup_seed(42)

        idx = []
        # The forward method
        from gpytorch.models import ApproximateGP

        class MultitaskGPModel(gpytorch.models.ExactGP):
                def __init__(self, train_x, train_y, likelihood):
                    super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
                    self.mean_module = gpytorch.means.ConstantMean()
                    self.covar_module = final_kernel
                    #self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-10., 10.)
                    self.feature_extractor = feature_extractor

                def forward(self, x): ##
                    projected_x, attention = self.feature_extractor(x)
                    projected_x = projected_x.squeeze()
                    projected_x = projected_x.reshape(-1)
                    mean_x = self.mean_module(projected_x)
                    covar_x = self.covar_module(projected_x)
                    return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        model = MultitaskGPModel(train_x, train_y, likelihood) 


        if torch.cuda.is_available():
            model = model.cuda()
            likelihood = likelihood.cuda()
            
        ##########################################################################################
        # setup_seed(42)

        # Training the model
        training_iterations = 500

        # Use the adam optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

        # "Loss" for GPs - the marginal log likelihood
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
        min_val_loss = 9999
        flag = 0
        retrain = 0
        torch.save(model.state_dict(), "./saved/Trigp/ETTh1.pth")
        original = model.covar_module
        #recorder[0] = 1
        #logger = get_logger('./log/'+'Trigp_ACSF1_'+ exp_name +'.log')

        def train():
            patience = 250
            counter = 0
            val_losses = []

            iterator = tqdm.notebook.tqdm(range(training_iterations))
            for i in iterator:
                # Find optimal model hyperparameters
                model.train()
                likelihood.train()

                # Zero backprop gradients
                optimizer.zero_grad()
                # Get output from model
                outputs = model(train_x)
                # Calc loss and backprop derivatives
                train_loss = -mll(outputs, train_y)
                train_loss.backward()
                iterator.set_postfix(train_loss=train_loss.item())
                optimizer.step()
                #logger.info('Epoch:[{}/{}]\t loss={:.5f}'.format(i , training_iterations, train_loss))

                model.eval()
                likelihood.eval()
                with torch.no_grad():
                    preds = model(val_x)
                    val_loss = -mll(preds, val_y)
                    val_losses.append(val_loss)

                global min_val_loss
                if min_val_loss > val_loss and val_loss > 0:
                    min_val_loss = val_loss
                    global flag
                    global original
                    original = model.covar_module
                    flag = 1
                    print(min_val_loss)
                    if retrain == 1:
                        print("Saving...")
                        torch.save(model.state_dict(), "./saved/Trigp/ETTh1.pth")
                    counter = 0
                else:
                    counter += 1
                    if counter % 10 == 0:
                        for params in optimizer.param_groups:
                            params['lr'] *= 0.9
                            print(params['lr'])

                if counter == patience:
                    break




        #logger.info('start training!')
        
        train()
        
        #logger.info('finish training!')
        
        ##########################################################################################
        ## Retrain:
        retrain = 1
        constrains = []
        for constraint_name, constraint in model.named_constraints():
            constrains.append(constraint)


        index = 0
        d = len(base_kernel_set)
        a1 = len(base_kernel_set)
        an = a1+(R-1)*d
        length = int((a1+an)*R/2)
        kernel_weights = torch.zeros(length)
        for param_name, param in model.named_parameters():
            if 'covar_module' and 'raw_outputscale' in param_name:
                #print(f'Parameter name: {param_name:42}')
                param_name = param_name.split('.')
                constrain = constrains[index]
                kernel_weights[index] = constraint.transform(param)
                index += 1
                #print(constraint.transform(param))

        pre = 0
        now = a1
        numbers = 1
        best_kernel_index = torch.zeros((R,numbers)).int()
        for r in range(R):
            weights = kernel_weights[pre:pre+d]
            pre = now
            now = a1 + a1+(r+1)*d
            weights = nn.functional.softmax(weights, dim=0)
            _, kernel_index = torch.sort(weights, descending=True)
            best_kernel_index[r] = kernel_index[:numbers]

        #print(best_kernel_index)
        ####################################################################################

        final_kernel = 0
        for i in range(R):
            add_kernel = copy.deepcopy(base_kernel_set[best_kernel_index[i][0]])
            for j in range(1, numbers):
                add_kernel += copy.deepcopy(base_kernel_set[best_kernel_index[i][j]])

            if i == 0:
                mul_kernel = copy.deepcopy(add_kernel)
                final_kernel = copy.deepcopy(mul_kernel)
                continue
            else:
                mul_kernel *= copy.deepcopy(add_kernel)
                final_kernel += copy.deepcopy(mul_kernel)


        print(final_kernel)
        
        min_val_loss = 9999
        flag = 0
        model.covar_module = final_kernel.cuda()
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
        #torch.save(model.state_dict(), "./saved/Trigp/ETTh1.pth")
        train()
        
        
        
        ##########################################################################################
        import warnings
        warnings.filterwarnings("ignore")
        import seaborn as sns
        setup_seed(42)

        from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

        #model.load_state_dict(torch.load("./saved/Trigp/ETTh1.pt"))

        # Making Predictions
        MAE = []
        RMSE = []
        MAPE = []
        model.load_state_dict(torch.load("./saved/Trigp/ETTh1.pth"))
        model.eval()
        likelihood.eval()

        with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
            for i in range(pre_horizon):
                if i == 0:
                    test_x = test[i:-pre_horizon+i,:seq_len].clone()
                test_y = test[i:-pre_horizon+i,seq_len:,].squeeze().clone()

                preds = model(test_x)
                real_preds = preds.mean
                real_test_y = test_y
                real_test_y = real_test_y

                real_preds = real_preds*std+mean
                real_test_y = real_test_y*std+mean
                print(real_preds.shape)
                print(real_test_y.shape)
                rmse = mean_squared_error(real_test_y.cpu(), real_preds.cpu())
                mae = mean_absolute_error(real_test_y.cpu(), real_preds.cpu())
                mape = mean_absolute_percentage_error(real_test_y.cpu(), real_preds.cpu())
                MAE.append(mae)
                RMSE.append(rmse)
                MAPE.append(mape)

                if i == pre_horizon-1:
                    break
                test_x = torch.cat([test_x.squeeze().clone(), preds.mean.unsqueeze(-1)], axis=1)
                test_x = test_x[:,1:].unsqueeze(-1).clone()
                '''
                new_window = test_x[-1].clone()
                new_window[:-1,] = new_window[1:,].clone()
                new_window[-1,] = preds.mean[-1]
                new_window = new_window.unsqueeze(0)
                test_x = torch.cat([test_x, new_window], 0)
                test_x = test_x[1:]
                '''

        mae = np.array(MAE).mean()
        rmse = np.array(RMSE).mean()
        mape = np.array(MAPE).mean()
        print('Test predict MAE, RMSE, MAPE:', mae, rmse, mape)
        results[order][0] = mae
        results[order][1] = rmse**0.5
        results[order][2] = mape
        #model_path = "./model_and_weights/M4/trigp/" + item + ".pkl"
        #torch.save(backend.state_dict(), model_path)
        order += 1
        

print(results)
print(np.mean(results, axis=0))
#print(recorder)


In [ ]:
print(results)
print(np.mean(results, axis=0))